In [11]:
from env import user, password, host 
from acquire import acquire_data

import pandas as pd
import os.path

The data is stored on a sql database. We'll use an env.py with the credintials needed to pull the data from sql. We'll pull the data into a pandas dataframe, which will be the primary datastructure for housing our data.

# Getting the Data from SQL

In [12]:
query = """
SELECT  bedroomcnt, 
		bathroomcnt,
        calculatedfinishedsquarefeet,
		taxvaluedollarcnt,
        taxamount,
		fips
FROM properties_2017 as p
JOIN predictions_2017 as pr USING(`parcelid`)
WHERE propertylandusetypeid = 261 and (transactiondate >= '2017-05-01' AND transactiondate <= '2017-06-31')
"""

zillow_url = f'mysql+pymysql://{user}:{password}@{host}/zillow'

df = pd.read_sql(query, zillow_url)

In [13]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,fips
0,3.0,2.0,1458.0,136104.0,2319.90,6037.0
1,2.0,1.0,1421.0,35606.0,543.69,6037.0
2,4.0,3.0,2541.0,880456.0,9819.72,6059.0
3,3.0,2.0,1650.0,614000.0,7673.19,6037.0
4,2.0,1.0,693.0,274237.0,3267.47,6037.0


# Dealing with Counties

### Importing an external dataset that deals with FIPS, which can tell us which county the residence is in

In [14]:
fips = pd.read_table('FIPS.txt')

In [15]:
fips.head()

,FIPS,Name,State
0,1001,Autauga,AL
1,1003,Baldwin,AL
2,1005,Barbour,AL
3,1007,Bibb,AL
4,1009,Blount,AL


# Merging the Data

In [16]:
df = pd.merge(left=df, right=fips, left_on='fips', right_on='FIPS')

In [7]:
df.head()

,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,fips,FIPS,Name,State
0,3.0,2.0,1458.0,136104.0,2319.90,6037.0,6037,Los Angeles,CA
1,2.0,1.0,1421.0,35606.0,543.69,6037.0,6037,Los Angeles,CA
2,3.0,2.0,1650.0,614000.0,7673.19,6037.0,6037,Los Angeles,CA
3,2.0,1.0,693.0,274237.0,3267.47,6037.0,6037,Los Angeles,CA
4,0.0,0.0,1378.0,168828.0,2135.39,6037.0,6037,Los Angeles,CA


# Generating a CSV to Reduce Stress on SQL Database

In [8]:
if os.path.isfile('zillow.csv'):
    print ("CSV already generated.")
else:
    df.to_csv('zillow.csv')
    print('Generated csv within zillow-project directory')

CSV already generated.


# Testing the Module

In [9]:
df = acquire_data()

csv has been previously generated.
Data Acquired


In [10]:
df.head()

,Unnamed: 0,bedroomcnt,bathroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,taxamount,fips,FIPS,Name,State
0,0,3.0,2.0,1458.0,136104.0,2319.90,6037.0,6037,Los Angeles,CA
1,1,2.0,1.0,1421.0,35606.0,543.69,6037.0,6037,Los Angeles,CA
2,2,3.0,2.0,1650.0,614000.0,7673.19,6037.0,6037,Los Angeles,CA
3,3,2.0,1.0,693.0,274237.0,3267.47,6037.0,6037,Los Angeles,CA
4,4,0.0,0.0,1378.0,168828.0,2135.39,6037.0,6037,Los Angeles,CA
